In [ ]:
import pprint as pp
import numpy as np
import time
import os
import sys
import random


import matplotlib.pyplot as plt
%matplotlib inline 
import matplotlib
from matplotlib.patches import Patch

import shutil
from numpy import genfromtxt
import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
from torch.autograd import Variable

torch.set_default_dtype(torch.float64)

import itertools
import numpy.linalg  as lin

import cProfile, pstats

from collections import OrderedDict
torch.set_num_threads(1) #cpu num
from fractions import Fraction
import seaborn as sns
import pandas as pd
import math
from collections import defaultdict
from matplotlib.lines import Line2D

In [ ]:
def fillup(_epp,_ep,_l):
    
    for ind in range(1,len(_epp)):
        if _epp[ind] not in _ep:
            _l.insert(ind,_l[ind-1])
    return _l    

def runmax(_l):
    
    return [max(_l[:i+1]) for i in range(len(_l))]

In [ ]:
%cd ../Logs

In [ ]:
logfolder = '%s/reg/%s/'

In [ ]:
DNAME = ['ijcnn1','rcv1','real-sim','news20','covtype']
ENDEP = [20,30,20,40,20]
TICK=[5,10,5,10,5]
ALGO_NAME = ['AI-SARAH', 'SARAH', 'SARAH+', 'SVRG', 'Adam', 'SGD w/m']

In [ ]:
mu = [2e-5,4.94e-5,1.844e-5,6.668e-5,2.295e-6]
BS = 64
TrainSize = [49990,20242,54231,14997,435759]

In [ ]:
df={}
for dd in range(len(DNAME)):
    dname = DNAME[dd]
    mu_d = mu[dd]
    perEpoch = TrainSize[dd]//BS
    DF = []
    for al in range(len(ALGO_NAME)):
        alg_name = ALGO_NAME[al]
        temp_logfolder = logfolder%(dname,algo)

        if alg_name in ['AI-SARAH','SVRG','SARAH','SARAH+']:
            tpp=[]
            for f in os.listdir(temp_logfolder): 
                if '.tar' not in f or 'DONE' in f or 'RUN' in f:
                    continue
                temp = torch.load(temp_logfolder+f)
                stat = np.array(temp['stat'])
                ter = list(stat[:,3])
                ter = [np.sum(ter[:oi]) for oi in range(1,len(ter)+1)]
                tpp+=list(np.round(ter,2))
                TPP = sorted(list(set(tpp)))
        else:
            tpp=[]
            for f in os.listdir(temp_logfolder): 
                if '.tar' not in f or 'DONE' in f or 'RUN' in f:
                    continue
                temp = torch.load(temp_logfolder+f)
                stat = np.array(temp['stat'])
                ter = list(stat[:,1])
                ter = [np.sum(ter[:oi]) for oi in range(1,len(ter)+1)]
                tpp+=list(np.round(ter,2))
                TPP = sorted(list(set(tpp)))
        
        for f in os.listdir(temp_logfolder):
            
            if '.tar' not in f or 'DONE' in f or 'RUN' in f:
                continue
            
            temp = torch.load(temp_logfolder+f)
            parm = temp['parm']
            
            if alg_name == 'AI-SARAH':
                lr = -1
                bs = parm[0]
                seed = parm[1]
                gamma = parm[2]
                if gamma != 1/32:
                    continue
                schedule = -1
                m = -1
            if alg_name == 'SARAH+':
                lr = parm[2]
                bs = parm[0]
                seed = parm[1]
                gamma = parm[3]
                schedule = -1
                m = -1
            if alg_name=='SARAH':
                lr = parm[2]
                bs = parm[0]
                seed = parm[1]
                gamma = -1
                schedule = -1
                m = parm[3]
            if alg_name=='Adam':
                temp_parm = f.split('-')
                lr = temp_parm[1]
                bs = parm[0]
                seed = parm[1]
                gamma = -1
                schedule = temp_parm[3]
                m = -1
            if alg_name=='SGD w/m':
                temp_parm = f.split('-')
                lr = temp_parm[1]
                bs = parm[0]
                seed = parm[1]
                gamma = -1
                schedule = temp_parm[3]
                m = -1
            if alg_name=='SVRG':
                lr = parm[2]
                bs = parm[0]
                seed = parm[1]
                gamma = -1
                schedule = -1
                m = parm[3]
            hist = np.array(temp['hist'])
            stat = np.array(temp['stat'])
            
            if alg_name in ['Adam','SGD w/m']:
                tp = list(stat[:,1])
                tp = np.round([np.sum(tp[:oi]) for oi in range(1,len(tp)+1)],2)
            else:
                tp = list(stat[:,3])
                tp = np.round([np.sum(tp[:oi]) for oi in range(1,len(tp)+1)],2)
            
            loss = list(hist[:,1])
            grad = list(hist[:,2])
            test = list(hist[:,3])
            ep = list(np.round(hist[:,0]))

            loss = fillup(TPP,tp,loss)
            grad = fillup(TPP,tp,grad)
            test = fillup(TPP,tp,test)
            ep = fillup(TPP,tp,ep)
            test = runmax(test)
            
            DF+=[[alg_name,bs,seed,lr,schedule,gamma,m,\
                  tp_i,li,gi,ti,ep_i] for tp_i,li,gi,ti,ep_i in \
                zip(TPP,loss,grad,test,ep)]
            
    df[dname]=pd.DataFrame(data=DF,columns=['algo','bs','seed','lr','schedule',\
                                            'gamma','m','tp',\
                                            'loss','grad','test','ep'])

In [ ]:
df2={}
for dd in range(len(DNAME)):
    dname = DNAME[dd]
    mu_d = mu[dd]
    perEpoch = TrainSize[dd]//BS
    DF = []
    for al in range(len(ALGO_NAME)):
        algo = ALGO[al]
        alg_name = ALGO_NAME[al]
        temp_logfolder = logfolder%(dname,algo)        
        if alg_name in ['AI-SARAH','SVRG','SARAH','SARAH+']:
            gpp=[]
            for f in os.listdir(temp_logfolder): 
                if '.tar' not in f or 'DONE' in f or 'RUN' in f:
                    continue
                temp = torch.load(temp_logfolder+f)
                stat = np.array(temp['stat'])               
                outer = list(stat[:,1])
                inner = list(stat[:,2])
                if alg_name=='AI-SARAH':
                    ter \
                    = np.array([(outer_i+inner_i*4)/perEpoch for outer_i,inner_i in zip(outer,inner)])
                else:
                    ter \
                    = np.array([(outer_i+inner_i*2)/perEpoch for outer_i,inner_i in zip(outer,inner)])
                gpp+=list(np.round(ter))
                GPP = sorted(list(set(gpp)))
        else:
            gpp=[]
            for f in os.listdir(temp_logfolder): 
                if '.tar' not in f or 'DONE' in f or 'RUN' in f:
                    continue
                temp = torch.load(temp_logfolder+f)
                stat = np.array(temp['stat'])
                gpp+=list(np.round(stat[:,0]))
                GPP = sorted(list(set(gpp)))
        
        for f in os.listdir(temp_logfolder):
            
            if '.tar' not in f or 'DONE' in f or 'RUN' in f:
                continue
            
            temp = torch.load(temp_logfolder+f)
            parm = temp['parm']
            
            if alg_name == 'AI-SARAH':
                lr = -1
                bs = parm[0]
                seed = parm[1]
                gamma = parm[2]
                if gamma != 1/32:
                    continue
                schedule = -1
                m = -1
            if alg_name == 'SARAH+':
                lr = parm[2]
                bs = parm[0]
                seed = parm[1]
                gamma = parm[3]
                schedule = -1
                m = -1
            if alg_name=='SARAH':
                lr = parm[2]
                bs = parm[0]
                seed = parm[1]
                gamma = -1
                schedule = -1
                m = parm[3]
            if alg_name=='Adam':
                temp_parm = f.split('-')
                lr = temp_parm[1]
                bs = parm[0]
                seed = parm[1]
                gamma = -1
                schedule = temp_parm[3]
                m = -1
            if alg_name=='SGD w/m':
                temp_parm = f.split('-')
                lr = temp_parm[1]
                bs = parm[0]
                seed = parm[1]
                gamma = -1
                schedule = temp_parm[3]
                m = -1
            if alg_name=='SVRG':
                lr = parm[2]
                bs = parm[0]
                seed = parm[1]
                gamma = -1
                schedule = -1
                m = parm[3]
            print('algo: %s, BS: %s, seed: %s, LR: %s, Sche: %s, gamma: %s, m: %s'\
                  %(alg_name,bs,seed,lr,schedule,gamma,m))
            
            hist = np.array(temp['hist'])
            stat = np.array(temp['stat'])
            
            if alg_name in ['Adam','SGD w/m']:
                gp = list(np.round(stat[:,0]))
            else:
                outer = list(stat[:,1])
                inner = list(stat[:,2])
                if alg_name=='AI-SARAH':
                    ter \
                    = np.array([(outer_i+inner_i*4)/perEpoch for outer_i,inner_i in zip(outer,inner)])
                else:
                    ter \
                    = np.array([(outer_i+inner_i*2)/perEpoch for outer_i,inner_i in zip(outer,inner)])
                gp=list(np.round(ter))
            loss = list(hist[:,1])
            grad = list(hist[:,2])
            test = list(hist[:,3])
            ep = list(np.round(hist[:,0]))
            loss = fillup(GPP,gp,loss)
            grad = fillup(GPP,gp,grad)
            test = fillup(GPP,gp,test)
            ep = fillup(GPP,gp,ep)
            test = runmax(test)
            
            DF+=[[alg_name,bs,seed,lr,schedule,gamma,m,\
                  gp_i,li,gi,ti,ep_i] for gp_i,li,gi,ti,ep_i in \
                zip(GPP,loss,grad,test,ep)]
            
    df2[dname]=pd.DataFrame(data=DF,columns=['algo','bs','seed','lr','schedule',\
                                            'gamma','m','gp',\
                                            'loss','grad','test','ep'])  

# Legend for Figures 4,5,6

In [ ]:
palette = sns.color_palette("tab10")[:6]
markers = ('o', 'v', '^', '<', '>', '8')
hue_order = ['AI-SARAH','SARAH','SARAH+','SVRG','Adam','SGD w/m']
handles=[Line2D([0],[0],color=palette[i],linewidth=3,dashes=[2,2],marker=markers[i],markersize=10)\
        for i in range(len(hue_order))]
fig,ax=plt.subplots(1,1,figsize=(15,1))
ax.legend(handles=handles,labels=hue_order,fancybox=True,fontsize=10,loc='center',\
         ncol=len(hue_order),prop={'size': 15})
ax.axis('off')
fig.tight_layout()
plt.show()

# Figure 4

In [ ]:
sns.set()
sns.set(context="paper",font='serif')
sns.set_style("ticks")
fig, ax = plt.subplots(1,len(DNAME),figsize=(3*len(DNAME),3),sharex='col',sharey=False)
palette = sns.color_palette("tab10")[:6]
markers = ('o', 'v', '^', '<', '>', '8')
dashes = [(2,2)]*6

for d in range(len(DNAME)):
    subdf=df2[DNAME[d]]
    subdf=subdf[subdf['ep']<=ENDEP[d]]
    lp=sns.lineplot(x='ep',y='grad',dashes=dashes,palette=palette,\
                 markers=markers,\
                 zorder=-1,markevery=5,markersize=8,lw=2.0,
                    legend=False,hue='algo',ci=95,style='algo',\
                    data=subdf,ax=ax[d])
    ax[d].set_ylabel('')
    ax[d].grid('both')
    ax[d].set_yscale('log')
for d in range(len(DNAME)):
    ax[d].set_xlabel('Effective Pass',fontsize=12)
    ax[d].set_title(DNAME[d],fontsize=15)
ax[0].set_ylabel(r'$||\nabla P(w)||^2$',fontsize=12)
fig.tight_layout()
plt.show()

In [ ]:
sns.set()
sns.set(context="paper",font='serif')
sns.set_style("ticks")
fig, ax = plt.subplots(1,len(DNAME),figsize=(3*len(DNAME),3),sharex='col',sharey=False)
palette = sns.color_palette("tab10")[:6]
markers = ('o', 'v', '^', '<', '>', '8')
dashes = [(2,2)]*6

for d in range(len(DNAME)):
    subdf=df[DNAME[d]]
    subdf=subdf[subdf['ep']<=ENDEP[d]]
    lp=sns.lineplot(x='tp',y='grad',dashes=dashes,palette=palette,\
                 markers=markers,\
                 zorder=-1,markevery=40,markersize=8,lw=2.0,
                    legend=False,hue='algo',ci=95,style='algo',\
                    data=subdf,ax=ax[d])
    ax[d].set_ylabel('')
    ax[d].grid('both')
    ax[d].set_yscale('log')
handles, labels = ax[0].get_legend_handles_labels()
for d in range(len(DNAME)):
    ax[d].set_xlabel('Wall Clock (sec)',fontsize=12)
ax[0].set_ylabel(r'$||\nabla P(w)||^2$',fontsize=12)
fig.tight_layout()
plt.show()

# Figure 5

In [ ]:
sns.set()
sns.set(context="paper",font='serif')
sns.set_style("ticks")
fig, ax = plt.subplots(1,len(DNAME),figsize=(3*len(DNAME),3),sharex='col',sharey=False)
loss_limit=[[0.181,0.3],[0.2,0.3],[0.155,0.3],[0.325,0.42],[0.5155,0.54]]
palette = sns.color_palette("tab10")[:6]
markers = ('o', 'v', '^', '<', '>', '8')
dashes = [(2,2)]*6

for d in range(len(DNAME)):
    subdf=df2[DNAME[d]]
    subdf=subdf[subdf['ep']<=ENDEP[d]]
        
    ylim = loss_limit[d]
    lp=sns.lineplot(x='ep',y='loss',dashes=dashes,palette=palette,\
                 markers=markers,\
                 zorder=-1,markevery=5,markersize=8,lw=2.0,
                    legend=False,hue='algo',ci=95,style='algo',\
                    data=subdf,ax=ax[d])
    ax[d].set_ylabel('')
    ax[d].grid('both')
    ax[d].set_ylim(ylim)
for d in range(len(DNAME)):
    ax[d].set_xlabel('Effective Pass',fontsize=12)
    ax[d].set_title(DNAME[d],fontsize=15)
    
ax[0].set_ylabel(r'$P(w)$',fontsize=12)
fig.tight_layout()
plt.show()

In [ ]:
sns.set()
sns.set(context="paper",font='serif')
sns.set_style("ticks")
fig, ax = plt.subplots(1,len(DNAME),figsize=(3*len(DNAME),3),sharex='col',sharey=False)
loss_limit=[[0.181,0.3],[0.2,0.3],[0.155,0.3],[0.325,0.42],[0.5155,0.54]]
palette = sns.color_palette("tab10")[:6]
markers = ('o', 'v', '^', '<', '>', '8')
dashes = [(2,2)]*6
for d in range(len(DNAME)):
    subdf=df[DNAME[d]]
    subdf=subdf[subdf['ep']<=ENDEP[d]]   
    ylim = loss_limit[d]
    lp=sns.lineplot(x='tp',y='loss',dashes=dashes,palette=palette,\
                 markers=markers,\
                 zorder=-1,markevery=40,markersize=8,lw=2.0,
                    legend=False,hue='algo',ci=95,style='algo',\
                    data=subdf,ax=ax[d])
    ax[d].set_ylabel('')
    ax[d].grid('both')
    ax[d].set_ylim(ylim)
for d in range(len(DNAME)):
    ax[d].set_xlabel('Wall Clock (sec)',fontsize=12)
ax[0].set_ylabel(r'$P(w)$',fontsize=12)
fig.tight_layout()
plt.show()
    
    

# Figure 6

In [ ]:
sns.set()
sns.set(context="paper",font='serif')
sns.set_style("ticks")
fig, ax = plt.subplots(1,len(DNAME),figsize=(3*len(DNAME),3),sharex='col',sharey=False)
test_limit = [[0.90,0.925],[0.94,0.96],[0.90,0.975],[0.90,0.937],[0.70,0.761]]
palette = sns.color_palette("tab10")[:6]
markers = ('o', 'v', '^', '<', '>', '8')
dashes = [(2,2)]*6
for d in range(len(DNAME)):
    subdf=df2[DNAME[d]]
    subdf=subdf[subdf['ep']<=ENDEP[d]]
    ylim = test_limit[d]
    lp=sns.lineplot(x='ep',y='test',dashes=dashes,palette=palette,\
                 markers=markers,\
                 zorder=-1,markevery=5,markersize=8,lw=2.0,
                    legend=False,hue='algo',ci=95,style='algo',\
                    data=subdf,ax=ax[d])
    ax[d].set_ylabel('')
    ax[d].grid('both')
    ax[d].set_ylim(ylim)
for d in range(len(DNAME)):
    ax[d].set_xlabel('Effective Pass',fontsize=12)
    ax[d].set_title(DNAME[d],fontsize=15)
ax[0].set_ylabel('Accuracy',fontsize=12)
fig.tight_layout()
plt.show()

In [ ]:
sns.set()
sns.set(context="paper",font='serif')
sns.set_style("ticks")
fig, ax = plt.subplots(1,len(DNAME),figsize=(3*len(DNAME),3),sharex='col',sharey=False)
test_limit = [[0.90,0.925],[0.94,0.96],[0.90,0.975],[0.90,0.937],[0.70,0.761]]
palette = sns.color_palette("tab10")[:6]
markers = ('o', 'v', '^', '<', '>', '8')
dashes = [(2,2)]*6
for d in range(len(DNAME)):
    subdf=df[DNAME[d]]
    subdf=subdf[subdf['ep']<=ENDEP[d]]
    ylim = test_limit[d]
    lp=sns.lineplot(x='tp',y='test',dashes=dashes,palette=palette,\
                 markers=markers,\
                 zorder=-1,markevery=40,markersize=8,lw=2.0,
                    legend=False,hue='algo',ci=95,style='algo',\
                    data=subdf,ax=ax[d])
    ax[d].set_ylabel('')
    ax[d].grid('both')
    ax[d].set_ylim(ylim)
for d in range(len(DNAME)):
    ax[d].set_xlabel('Wall Clock (sec)',fontsize=12)    
ax[0].set_ylabel('Accuracy',fontsize=12)
fig.tight_layout()
plt.show() 

In [ ]:
exit(0)